In [78]:
import pandas as pd
import numpy as np
import os
from collections import deque

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

# Day 3 Gear Ratios
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..\
...*......\
..35..633.\
......#...\
617*......\
.....+.58.\
..592.....\
......755.\
...$.*....\
.664.598..\

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [79]:
with open('input.txt', 'r') as file:
    input = file.read()

# input = '''467..114..
# ...*......
# ..35..633.
# ......#...
# 617*......
# .....+.58.
# ..592.....
# ......755.
# ...$.*....
# .664.598..'''

DF = pd.DataFrame(pd.Series(input.split('\n')).apply(list).to_list()).dropna()

DF = pd.concat([pd.Series(['.']*len(DF), name='StartDot'), DF], axis=1)
DF = pd.concat([DF, pd.Series(['.']*len(DF), name='EndDot')], axis=1)

# Add a row with '.' to the first and last positions
new_row = pd.DataFrame([['.']*len(DF.columns)], columns=DF.columns)
DF = pd.concat([new_row, DF]).reset_index(drop=True)
DF = pd.concat([DF, new_row]).reset_index(drop=True)
DF.columns = range(len(DF.columns))

PART_NUMBERS = {
    'PartAnchor': [], 
    'PartNumber': [], 
    'CenterAnchor': [],
    'CenterSymbol': []
}

In [80]:
class Tile:
    def __init__(self, i, j):
        self.i = i
        self.j = j

    def get(self):
        return DF.iloc[self.i, self.j]
    
    def check_number(self):
        if self.get().isdigit():
            return True
        return False
    
## create a class to manage checking surroundings
class Map:
    def __init__(self, i, j):
        self.c = Tile(i, j)
        self.r = Tile(i, j+1)
        self.l = Tile(i, j-1)
        self.u = Tile(i-1, j)
        self.d = Tile(i+1, j)
        self.ur = Tile(i-1, j+1)
        self.ul = Tile(i-1, j-1)
        self.dr = Tile(i+1, j+1)
        self.dl = Tile(i+1, j-1)
    

def add_part_number(m: Map, center_map: Map):
    part_dqueue = deque([m.c.get()])
    temp_m = Map(m.l.i, m.l.j)

    # check left tiles
    while temp_m.c.check_number():
        part_dqueue.appendleft(temp_m.c.get())
        temp_m = Map(temp_m.l.i, temp_m.l.j)
    
    temp_m = Map(m.r.i, m.r.j)

    # check right tiles
    while temp_m.c.check_number():
        part_dqueue.append(temp_m.c.get())
        temp_m = Map(temp_m.r.i, temp_m.r.j)

    part_anchor = f'{temp_m.l.i}_{temp_m.l.j}'
    part_number = int(''.join(map(str, part_dqueue)))
    center_anchor = f'{center_map.c.i}_{center_map.c.j}'
    center_symbol = center_map.c.get()

    if part_anchor not in PART_NUMBERS['PartAnchor']:
        print(f'Found Part Number -> ({part_anchor}): {part_number}')

        PART_NUMBERS['PartAnchor'].append(part_anchor)
        PART_NUMBERS['PartNumber'].append(part_number)
        PART_NUMBERS['CenterAnchor'].append(center_anchor)
        PART_NUMBERS['CenterSymbol'].append(center_symbol)

    
def check_around_map(m: Map):
    try:
        if m.r.check_number():
            add_part_number(Map(m.r.i, m.r.j), m)
    except IndexError:
        pass

    try:
        if m.l.check_number():
            add_part_number(Map(m.l.i, m.l.j), m)
    except IndexError:
        pass
    
    try:
        if m.u.check_number():
            add_part_number(Map(m.u.i, m.u.j), m)
    except IndexError:
        pass

    try:
        if m.d.check_number():
            add_part_number(Map(m.d.i, m.d.j), m)
    except IndexError:
        pass
    
    try:
        if m.ur.check_number():
            add_part_number(Map(m.ur.i, m.ur.j), m)
    except IndexError:
        pass

    try:
        if m.ul.check_number():
            add_part_number(Map(m.ul.i, m.ul.j), m)
    except IndexError:
        pass
    
    try:
        if m.dl.check_number():
            add_part_number(Map(m.dl.i, m.dl.j), m)
    except IndexError:
        pass

    try:
        if m.dr.check_number():
            add_part_number(Map(m.dr.i, m.dr.j), m)
    except IndexError:
        pass

def check_current_map(m: Map):
    if m.c.get() != '.' and not m.c.get().isdigit():
        return True
    return False



In [81]:
for i in range(len(DF)):
    for j in range(len(DF.columns)):
        print(i,j)
        current_map = Map(i, j)
        
        # check if we are at a special character
        if check_current_map(current_map):
            check_around_map(current_map)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 5

In [82]:
sum(PART_NUMBERS['PartNumber'])

528799

Answer - 528799

## Part 2
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..\
...*......\
..35..633.\
......#...\
617*......\
.....+.58.\
..592.....\
......755.\
...$.*....\
.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

In [83]:
part_2_df = pd.DataFrame(PART_NUMBERS).query('CenterSymbol == "*"')
part_2_df

,PartAnchor,PartNumber,CenterAnchor,CenterSymbol
0,3_13,360,2_13,*
1,1_12,662,2_13,*
3,1_49,805,2_48,*
4,3_51,677,2_48,*
5,1_62,862,2_62,*
...,...,...,...,...
1056,138_85,155,139_83,*
1057,140_85,474,139_83,*
1061,138_116,620,139_117,*
1062,140_127,471,139_127,*


In [84]:
CenterAnchors = part_2_df['CenterAnchor'].value_counts().reset_index().query('count == 2')['CenterAnchor']

df = part_2_df.query('CenterAnchor.isin(@CenterAnchors)').copy()

df['counter'] = df.groupby('CenterAnchor').cumcount()

final = df.pivot(index='CenterAnchor', columns='counter', values='PartNumber').reset_index()
final.columns = ['CenterAnchor', 'Part1', 'Part2']

final['multiply'] = final['Part1']*final['Part2']

final['multiply'].sum()

84907174

Answer = 84907174